In [1]:
from IPython.display import display, Markdown

with open("restaurent_search.md", "r") as f:
    display(Markdown(f.read()))

## restaurant_search
- i'm looking for a place to eat
- I want to grab lunch
- I am searching for a dinner spot
- i'm looking for a place in the [north](location) of town
- show me [chinese](cuisine) restaurants
- show me [chines](cuisine:chinese) restaurants in the [north](location)
- show me a [mexican](cuisine) place in the [centre](location)
- i am looking for an [indian](cuisine) spot called olaolaolaolaolaola
- search for restaurants
- anywhere in the [west](location)
- anywhere near [18328](location)
- I am looking for [asian fusion](cuisine) food
- I am looking a restaurant in [29432](location)
- I am looking for [mexican indian fusion](cuisine)
- [central](location) [indian](cuisine) restaurant

In [2]:
from spacy_crfsuite import read_file

train_data = read_file("restaurent_search.md")
train_data

[{'text': "i'm looking for a place to eat", 'entities': []},
 {'text': 'I want to grab lunch', 'entities': []},
 {'text': 'I am searching for a dinner spot', 'entities': []},
 {'text': "i'm looking for a place in the north of town",
  'entities': [{'start': 31,
    'end': 36,
    'value': 'north',
    'entity': 'location'}]},
 {'text': 'show me chinese restaurants',
  'entities': [{'start': 8,
    'end': 15,
    'value': 'chinese',
    'entity': 'cuisine'}]},
 {'text': 'show me chines restaurants in the north',
  'entities': [{'start': 8,
    'end': 14,
    'value': 'chinese',
    'entity': 'cuisine'},
   {'start': 34, 'end': 39, 'value': 'north', 'entity': 'location'}]},
 {'text': 'show me a mexican place in the centre',
  'entities': [{'start': 10,
    'end': 17,
    'value': 'mexican',
    'entity': 'cuisine'},
   {'start': 31, 'end': 37, 'value': 'centre', 'entity': 'location'}]},
 {'text': 'i am looking for an indian spot called olaolaolaolaolaola',
  'entities': [{'start': 20,
  

In [3]:
import spacy

from spacy_crfsuite.crf_extractor import prepare_example
from spacy_crfsuite.tokenizer import SpacyTokenizer

nlp = spacy.load("en_core_web_sm")
tokenizer = SpacyTokenizer(nlp)

train_dataset = [prepare_example(ex, tokenizer=tokenizer) for ex in train_data]
train_dataset[0]

/Users/talmago/git/spacy_crfsuite/.venv/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[CRFToken(text='i', tag='PRP', entity='O', pattern={}, dense_features=[]),
 CRFToken(text="'m", tag='VBP', entity='O', pattern={}, dense_features=[]),
 CRFToken(text='looking', tag='VBG', entity='O', pattern={}, dense_features=[]),
 CRFToken(text='for', tag='IN', entity='O', pattern={}, dense_features=[]),
 CRFToken(text='a', tag='DT', entity='O', pattern={}, dense_features=[]),
 CRFToken(text='place', tag='NN', entity='O', pattern={}, dense_features=[]),
 CRFToken(text='to', tag='TO', entity='O', pattern={}, dense_features=[]),
 CRFToken(text='eat', tag='VB', entity='O', pattern={}, dense_features=[])]

In [4]:
import srsly

component_config = srsly.read_json("default-config.json")
component_config

{'features': [['low', 'title', 'upper'],
  ['low',
   'bias',
   'prefix5',
   'prefix2',
   'suffix5',
   'suffix3',
   'suffix2',
   'upper',
   'title',
   'digit'],
  ['low', 'title', 'upper']],
 'c1': 0.003,
 'c2': 0.03}

In [5]:
from spacy_crfsuite import CRFExtractor

crf_extractor = CRFExtractor(component_config=component_config)
crf_extractor

In [ ]:
crf_extractor.fine_tune(train_dataset, n_iter=100, random_state=42).best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/talmago/git/spacy_crfsuite/.venv/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
crf_extractor.component_config["c1"] = 0.003
crf_extractor.component_config["c2"] = 0.03
crf_extractor.train(train_dataset)

In [ ]:
# ! pip install sklearn

classification_report = crf_extractor.eval(train_dataset)
print(classification_report[1])

In [ ]:
print(crf_extractor.explain())

In [ ]:
from spacy_crfsuite.dense_features import DenseFeatures

example = {"text": "show mexican restaurents up north"}
prepare_example(example)
crf_extractor.process(example)

In [ ]:
import spacy

from spacy_crfsuite import CRFEntityExtractor

nlp = spacy.blank('en')
pipe = CRFEntityExtractor(nlp, crf_extractor=crf_extractor)
nlp.add_pipe(pipe)

doc = nlp("show mexican restaurents up north")
for ent in doc.ents:
    print(ent.text, "--", ent.label_)

In [ ]:
# Save model to disk ...
# crf_extractor.to_disk()